# Prompt Templates

In [1]:
from langchain_openai import OpenAI
from langchain_openai import ChatOpenAI

llm = OpenAI()
chat = ChatOpenAI()

c:\Projects\ai\langchain-playground\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:25: UserWarning: Core Pydantic V1 functionality isn't compatible with Python 3.14 or greater.
  from pydantic.v1.fields import FieldInfo as FieldInfoV1
c:\Projects\ai\langchain-playground\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
PyTorch was not found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


### Basic Usage

In [2]:
from langchain_core.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template('''
Traduza o texto do usuário para o {language}.
Texto:{text}
''')

#### Invoke prompt

In [3]:
answer = llm.invoke(prompt_template.format(language='inglês', text='Olá, mundo!'))
print(answer)


Hello, world!


### Composing Prompts

In [4]:
from langchain_core.prompts import ChatPromptTemplate

prompt_template_restriction = ChatPromptTemplate.from_template('''
Responda a pergunta do usuário usando até {max_words} palavras.
''')

prompt_template_translation = ChatPromptTemplate.from_template('''
Traduza o texto do usuário para o {language}.
''')

prompt_template_question = ChatPromptTemplate.from_template('''
Pergunta:
{question}
''')


#### Invoke prompt

In [5]:
prompt_template_final = prompt_template_restriction + prompt_template_translation + prompt_template_question

prompt_template_final_formatted = prompt_template_final.format(max_words=500, language='inglês', question='Onde fica o planeta terra?')

answer = llm.invoke(prompt_template_final_formatted)
print(answer)


Answer:
The planet Earth is located in the Milky Way galaxy, approximately 149.6 million kilometers away from the Sun.


### Chat Templates

#### Basic usage

In [6]:
from langchain_core.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template('Esta é a minha pergunta: {question}')

prompt_template_formatted = prompt_template.format_messages(question= 'Onde fica o planeta terra?')


#### Invoke prompt

In [7]:
answer = chat.invoke(prompt_template_formatted)
print(answer.content)

O planeta Terra está localizado no Sistema Solar, na terceira órbita a partir do Sol. Sua posição exata é de aproximadamente 150 milhões de quilômetros de distância do Sol, o que corresponde a uma distância média de aproximadamente 1 unidade astronômica (UA). Em relação à nossa galáxia, a Via Láctea, a Terra está localizada em um dos braços espirais, conhecido como Braço de Órion.


#### Chat Conversation

In [8]:
from langchain_core.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages([
    ('system', 'Você é um assistente de IA. E responde tudo com um toque de humor.'),
    ('user', 'Olá, tudo bem?'),
    ('assistant', 'Olá, melhor que nunca!'),
    ('user', '{input}')
])

#### Invoke prompt

In [9]:
answer = chat.invoke(chat_template.format_messages(input= 'Onde fica o planeta terra?'))
print(answer.content)

Bem na última vez que verifiquei, ele estava localizado em um sistema solar chamado Via Láctea. Mas sabemos como esses planetas adoram ficar se movendo por aí, então é bom conferir no GPS cósmico antes de sair de casa.


### Templates + Few-shot + LLM

#### Setup examples

In [10]:
examples = [
    {"pergunta": "Quem viveu mais tempo, Muhammad Ali ou Alan Turing?", 
     "resposta": 
     """São necessárias perguntas de acompanhamento aqui: Sim. 
Pergunta de acompanhamento: Quantos anos Muhammad Ali tinha quando morreu? 
Resposta intermediária: Muhammad Ali tinha 74 anos quando morreu. 
Pergunta de acompanhamento: Quantos anos Alan Turing tinha quando morreu? 
Resposta intermediária: Alan Turing tinha 41 anos quando morreu. 
Então a resposta final é: Muhammad Ali 
""", 
    }, 
    {"pergunta": "Quando nasceu o fundador do craigslist?", 
     "resposta": 
"""São necessárias perguntas de acompanhamento aqui: Sim. 
Pergunta de acompanhamento: Quem foi o fundador do craigslist? 
Resposta intermediária: O craigslist foi fundado por Craig Newmark. 
Pergunta de acompanhamento: Quando nasceu Craig Newmark? 
Resposta intermediária: Craig Newmark nasceu em 6 de dezembro de 1952. 
Então a resposta final é: 6 de dezembro de 1952 
""", 
    }, 
    {"pergunta": "Quem foi o avô materno de George Washington?",
     "resposta": 
"""São necessárias perguntas de acompanhamento aqui: Sim. 
Pergunta de acompanhamento: Quem foi a mãe de George Washington? 
Resposta intermediária: A mãe de George Washington foi Mary Ball Washington. 
Pergunta de acompanhamento: Quem foi o pai de Mary Ball Washington? 
Resposta intermediária: O pai de Mary Ball Washington foi Joseph Ball. 
Então a resposta final é: Joseph Ball 
""", 
    },
    {"pergunta": "Os diretores de Jaws e Casino Royale são do mesmo país?", 
     "resposta": 
"""São necessárias perguntas de acompanhamento aqui: Sim. 
Pergunta de acompanhamento: Quem é o diretor de Jaws? 
Resposta Intermediária: O diretor de Jaws é Steven Spielberg. 
Pergunta de acompanhamento: De onde é Steven Spielberg? 
Resposta Intermediária: Estados Unidos. 
Pergunta de acompanhamento: Quem é o diretor de Casino Royale? 
Resposta Intermediária: O diretor de Casino Royale é Martin Campbell. 
Pergunta de acompanhamento: De onde é Martin Campbell? 
Resposta Intermediária: Nova Zelândia. 
Então a resposta final é: Não 
""",
    },
]

#### Setup template

In [11]:
from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate

example_prompt = PromptTemplate(
    input_variables=['pergunta', 'resposta'],
    template='Pergunta {pergunta}\n{resposta}'
)

example_prompt.format(**examples[0])

'Pergunta Quem viveu mais tempo, Muhammad Ali ou Alan Turing?\nSão necessárias perguntas de acompanhamento aqui: Sim. \nPergunta de acompanhamento: Quantos anos Muhammad Ali tinha quando morreu? \nResposta intermediária: Muhammad Ali tinha 74 anos quando morreu. \nPergunta de acompanhamento: Quantos anos Alan Turing tinha quando morreu? \nResposta intermediária: Alan Turing tinha 41 anos quando morreu. \nEntão a resposta final é: Muhammad Ali \n'

#### Invoke prompt

In [12]:
prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix='Pergunta: {input}',
    input_variables=['input']
)

prompt_formatted = prompt.format(input='Quem fez mais gols, Romário ou Pelé?')
print(prompt_formatted)

llm.invoke(prompt_formatted)

Pergunta Quem viveu mais tempo, Muhammad Ali ou Alan Turing?
São necessárias perguntas de acompanhamento aqui: Sim. 
Pergunta de acompanhamento: Quantos anos Muhammad Ali tinha quando morreu? 
Resposta intermediária: Muhammad Ali tinha 74 anos quando morreu. 
Pergunta de acompanhamento: Quantos anos Alan Turing tinha quando morreu? 
Resposta intermediária: Alan Turing tinha 41 anos quando morreu. 
Então a resposta final é: Muhammad Ali 


Pergunta Quando nasceu o fundador do craigslist?
São necessárias perguntas de acompanhamento aqui: Sim. 
Pergunta de acompanhamento: Quem foi o fundador do craigslist? 
Resposta intermediária: O craigslist foi fundado por Craig Newmark. 
Pergunta de acompanhamento: Quando nasceu Craig Newmark? 
Resposta intermediária: Craig Newmark nasceu em 6 de dezembro de 1952. 
Então a resposta final é: 6 de dezembro de 1952 


Pergunta Quem foi o avô materno de George Washington?
São necessárias perguntas de acompanhamento aqui: Sim. 
Pergunta de acompanhamento: 

'\nSão necessárias perguntas de acompanhamento aqui: Sim. \nPergunta de acompanhamento: Quantos gols Romário fez em sua carreira? \nResposta intermediária: Romário fez 772 gols em sua carreira. \nPergunta de acompanhamento: Quantos gols Pelé fez em sua carreira? \nResposta intermediária: Pelé fez 1283 gols em sua carreira. \nEntão a resposta final é: Pelé'

### Templates + Few-shot + Chat

#### Setup examples

In [13]:
from langchain_core.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain_core.prompts import ChatPromptTemplate

example_prompt = ChatPromptTemplate.from_messages(
    [('human', '{pergunta}'),
     ('ai', '{resposta}')]
)

print(example_prompt.format_messages(**examples[0]))

[HumanMessage(content='Quem viveu mais tempo, Muhammad Ali ou Alan Turing?', additional_kwargs={}, response_metadata={}), AIMessage(content='São necessárias perguntas de acompanhamento aqui: Sim. \nPergunta de acompanhamento: Quantos anos Muhammad Ali tinha quando morreu? \nResposta intermediária: Muhammad Ali tinha 74 anos quando morreu. \nPergunta de acompanhamento: Quantos anos Alan Turing tinha quando morreu? \nResposta intermediária: Alan Turing tinha 41 anos quando morreu. \nEntão a resposta final é: Muhammad Ali \n', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[])]


#### Setup template

In [14]:
few_shot_template = FewShotChatMessagePromptTemplate(
    examples=examples,
    example_prompt=example_prompt
)

prompt_template = ChatPromptTemplate.from_messages(
    [
    few_shot_template,
    ('human', '{input}')
    ]
)

prompt = prompt_template.format_messages(input='Quem fez mais gols, Romário ou Pelé?')
prompt



[HumanMessage(content='Quem viveu mais tempo, Muhammad Ali ou Alan Turing?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='São necessárias perguntas de acompanhamento aqui: Sim. \nPergunta de acompanhamento: Quantos anos Muhammad Ali tinha quando morreu? \nResposta intermediária: Muhammad Ali tinha 74 anos quando morreu. \nPergunta de acompanhamento: Quantos anos Alan Turing tinha quando morreu? \nResposta intermediária: Alan Turing tinha 41 anos quando morreu. \nEntão a resposta final é: Muhammad Ali \n', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[]),
 HumanMessage(content='Quando nasceu o fundador do craigslist?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='São necessárias perguntas de acompanhamento aqui: Sim. \nPergunta de acompanhamento: Quem foi o fundador do craigslist? \nResposta intermediária: O craigslist foi fundado por Craig Newmark. \nPergunta de acompanhamento: Quando nasceu Craig Newmark? \nRespo

#### Invoke prompt

In [15]:
answer = chat.invoke(prompt)
print(answer.content)

Pelé marcou mais gols ao longo de sua carreira do que Romário. Pelé é considerado um dos maiores artilheiros da história do futebol, tendo marcado mais de 1000 gols em jogos oficiais. Já Romário, apesar de também ter uma carreira impressionante, marcou menos gols do que Pelé.
